In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import pickle
import PIL
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
path ='../Data'
save_path = '../Preprocessed_Data'

In [3]:
def load_data(file_name):
    with open(os.path.join(save_path,file_name),'rb') as f:
        data = pickle.load(f)
    return data

In [4]:
# load the data
# train_images = load_data('resized_train_images.pickle')

In [5]:
with open("../Data/train.jsonl") as f:
    train_lines = f.readlines()

data_train = [eval(c) for c in train_lines]
df_train = pd.DataFrame(data_train)

with open("../Data/dev_seen.jsonl") as f:
    dev_seen_lines = f.readlines()

data_dev_seen = [eval(c) for c in dev_seen_lines]
df_dev_seen = pd.DataFrame(data_dev_seen)

with open("../Data/test_seen.jsonl") as f:
    test_seen_lines = f.readlines()

data_test_seen = [eval(c) for c in test_seen_lines]
df_test_seen = pd.DataFrame(data_test_seen)

In [6]:
df_train, _ = train_test_split(df_train, test_size=0.7, random_state=42, stratify=df_train['label'])

In [7]:
def load_images(df):
    images = []
    ls_exists = []
    paths = df['img'].to_list()
    for i in tqdm(range(len(paths))):
        if not os.path.exists(path + "/" + paths[i]):
            ls_exists.append(False)
            continue
        ls_exists.append(True)
        img = np.array(PIL.Image.open(path + "/" + paths[i]))
        images.append(img)
    return images, ls_exists

In [8]:
train_images, train_ls_exists = load_images(df_train)
# dev_seen_images, dev_seen_ls_exists = load_images(df_dev_seen)
# test_seen_images, test_seen_ls_exists = load_images(df_test_seen)

  0%|          | 0/2550 [00:00<?, ?it/s]

In [9]:
df_train['img_exists'] = train_ls_exists
# df_dev_seen['img_exists'] = dev_seen_ls_exists
# df_test_seen['img_exists'] = test_seen_ls_exists

df_train = df_train[df_train['img_exists'] == True]
# df_dev_seen = df_dev_seen[df_dev_seen['img_exists'] == True]
# df_test_seen = df_test_seen[df_test_seen['img_exists'] == True]

In [10]:
df_dev_seen

,id,img,label,text
0,08291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,03745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime
...,...,...,...,...
495,83675,img/83675.png,0,i'm gonna be like phelps one day
496,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...
497,48670,img/48670.png,0,look at this sandwich maker club i found on wi...
498,09863,img/09863.png,0,diverse group of women


In [11]:
# min_length = float('inf')
# min_breadth = float('inf')
# max_length = 0
# max_breadth = 0
# for i in range(len(train_images)):
#     if train_images[i].shape[0] < min_length:
#         min_length = train_images[i].shape[0]
#     if train_images[i].shape[1] < min_breadth:
#         min_breadth = train_images[i].shape[1]
#     if train_images[i].shape[0] > max_length:
#         max_length = train_images[i].shape[0]
#     if train_images[i].shape[1] > max_breadth:
#         max_breadth = train_images[i].shape[1]
# for i in range(len(dev_seen_images)):
#     if dev_seen_images[i].shape[0] < min_length:
#         min_length = dev_seen_images[i].shape[0]
#     if dev_seen_images[i].shape[1] < min_breadth:
#         min_breadth = dev_seen_images[i].shape[1]
#     if dev_seen_images[i].shape[0] > max_length:
#         max_length = dev_seen_images[i].shape[0]
#     if dev_seen_images[i].shape[1] > max_breadth:
#         max_breadth = dev_seen_images[i].shape[1]
# for i in range(len(test_seen_images)):
#     if test_seen_images[i].shape[0] < min_length:
#         min_length = test_seen_images[i].shape[0]
#     if test_seen_images[i].shape[1] < min_breadth:
#         min_breadth = test_seen_images[i].shape[1]
#     if test_seen_images[i].shape[0] > max_length:
#         max_length = test_seen_images[i].shape[0]
#     if test_seen_images[i].shape[1] > max_breadth:
#         max_breadth = test_seen_images[i].shape[1]
# print(min_length, min_breadth, max_length, max_breadth)

In [12]:
resnet_length = 224
resnet_breadth = 224

In [13]:
def resize_images(images, length, breadth):
    resized_images = []
    for i in range(len(images)):
        image_length = images[i].shape[0]
        image_breadth = images[i].shape[1]
        image_new = images[i]
        if image_length > length:
            image_new = image_new[:length, :, :]
        elif image_length < length:
            # repeat the image horizontally
            extra = length - image_length
            if extra < image_length:
                chunk = image_new[image_length - extra:, :, :]
            else:
                number_of_full_chunks = int(extra / image_length)
                remainder = extra % image_length
                chunk = image_new
                for j in range(number_of_full_chunks-1):
                    chunk = np.concatenate((chunk, image_new), axis=0)
                if remainder > 0:
                    chunk = np.concatenate((chunk, image_new[:remainder, :, :]), axis=0)
            image_new = np.concatenate((image_new, chunk), axis=0)
        if image_breadth > breadth:
            image_new = image_new[:, :breadth, :]
        elif image_breadth < breadth:
            # repeat the image vertically
            extra = breadth - image_breadth
            if extra < image_breadth:
                chunk = image_new[:, image_breadth - extra:, :]
            else:
                number_of_full_chunks = int(extra / image_breadth)
                remainder = extra % image_breadth
                chunk = image_new
                for j in range(number_of_full_chunks-1):
                    chunk = np.concatenate((chunk, image_new), axis=1)
                if remainder > 0:
                    chunk = np.concatenate((chunk, image_new[:, :remainder, :]), axis=1)
            image_new = np.concatenate((image_new, chunk), axis=1)
        resized_images.append(image_new)
    return resized_images

In [14]:
resized_train_images = resize_images(train_images, resnet_length, resnet_breadth)
# resized_dev_seen_images = resize_images(dev_seen_images, resnet_length, resnet_breadth)
# resized_test_seen_images = resize_images(test_seen_images, resnet_length, resnet_breadth)

In [15]:
# find image smaller than mean_length and mean_breadth
# for i in range(len(train_images)):
#     if train_images[i].shape[0] < resnet_length or train_images[i].shape[1] < resnet_breadth:
#         print(train_images[i].shape)
#         # plt.imshow(train_images[i])
#         plt.imshow(resized_train_images[i])
#         break

# for i in range(len(dev_seen_images)):
#     if dev_seen_images[i].shape[0] < resnet_length or dev_seen_images[i].shape[1] < resnet_breadth:
#         print(dev_seen_images[i].shape, resized_dev_seen_images[i].shape)
#         # plt.imshow(dev_seen_images[i])
#         plt.imshow(resized_dev_seen_images[i])
#         break

# for i in range(len(test_seen_images)):
#     if test_seen_images[i].shape[0] < resnet_length or test_seen_images[i].shape[1] < resnet_breadth:
#         print(test_seen_images[i].shape, resized_test_seen_images[i].shape)
#         # plt.imshow(test_seen_images[i])
#         plt.imshow(resized_test_seen_images[i])
#         break


In [16]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# Load the ResNet50 model
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

# Define a function to get the ResNet embeddings for an image
def get_resnet_embedding(image):
    # Convert the image to a PIL Image object
    image = Image.fromarray(image.astype('uint8'), 'RGB')

    # Load and preprocess the image
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0)

    # Get the ResNet embeddings from penultimate layer
    with torch.no_grad():
        embedding = resnet(image).squeeze()
        

    return embedding.numpy()

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
# # normalize the images
# def normalize_images(image):
#     return image / 255

In [18]:
resnet_embedding_train_images = []
for i in tqdm(range(len(resized_train_images))):
    resnet_embedding_train_images.append(get_resnet_embedding(resized_train_images[i]))

# resnet_embedding_dev_seen_images = []
# for i in tqdm(range(len(resized_dev_seen_images))):
#     resnet_embedding_dev_seen_images.append(get_resnet_embedding(resized_dev_seen_images[i]))

# resnet_embedding_test_seen_images = []
# for i in tqdm(range(len(resized_test_seen_images))):
#     resnet_embedding_test_seen_images.append(get_resnet_embedding(resized_test_seen_images[i]))

  0%|          | 0/1297 [00:00<?, ?it/s]

In [19]:
resnet_embedding_train_images[0].shape

(2048,)

In [20]:
# for i in range(len(resized_train_images)):
#     resized_train_images[i] = normalize_images(resized_train_images[i])
# for i in range(len(resized_dev_seen_images)):
#     resized_dev_seen_images[i] = normalize_images(resized_dev_seen_images[i])
# for i in range(len(resized_test_seen_images)):
#     resized_test_seen_images[i] = normalize_images(resized_test_seen_images[i])

In [21]:
with open(save_path + '/resnet_embedding_train_images.pkl', 'wb') as f:
    pickle.dump(resnet_embedding_train_images, f)

# with open(save_path + '/resnet_embedding_dev_seen_images.pkl', 'wb') as f:
#     pickle.dump(resnet_embedding_dev_seen_images, f)

# with open(save_path + '/resnet_embedding_test_seen_images.pkl', 'wb') as f:
#     pickle.dump(resnet_embedding_test_seen_images, f)

In [22]:
# with open(save_path + "/resized_train_images.pickle", "wb") as f:
#   pickle.dump(resized_train_images, f)

# with open(save_path + "/resized_dev_seen_images.pickle", "wb") as f:
#   pickle.dump(resized_dev_seen_images, f)

# with open(save_path + "/resized_test_seen_images.pickle", "wb") as f:
#   pickle.dump(resized_test_seen_images, f)

In [23]:
df_train.to_csv(save_path + "/df_train.csv", index=False)
# df_dev_seen.to_csv(save_path + "/df_dev_seen.csv", index=False)
# df_test_seen.to_csv(save_path + "/df_test_seen.csv", index=False)